In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, GridSearchCV
from sklearn.feature_extraction import DictVectorizer as DV

from sklearn.metrics import roc_auc_score

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier

from matplotlib import pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter('ignore', category=FutureWarning)

import warnings 
warnings.filterwarnings("ignore") 

Открываем данные:

In [2]:
data = pd.read_csv('train_data.csv')
labels = pd.read_csv('train_labels.csv')

Добавим функцию по удалению стоблцов в которых все строки пустые

In [3]:
def get_nan_cols(df, nan_percent=0.8):
    threshold = len(df.index) * nan_percent
    return [c for c in df.columns if sum(df[c].isnull()) >= threshold]

Отделяем столбцы с количественными и категориальными переменными согласно условиям задания

In [4]:
numeric_cols = data.columns.values[:190]
categorical_cols = list(set(data.columns.values.tolist()) - set(numeric_cols))

Запускаем функцию по удалению стоблцов в которых все строки пустые

In [5]:
X_real = data[numeric_cols].drop(data[numeric_cols][get_nan_cols(data[numeric_cols], 1)].columns, axis = 1)
X_cat = data[categorical_cols].drop(data[categorical_cols][get_nan_cols(data[categorical_cols], 1)].columns, axis = 1)

Работаем с пустыми строками. Количественные признаки будет рассматривать с заменой на '0' и на среднее значение по столбцу. Категориальные - делаем замену на 'NA'.

In [6]:
X_real_zeros = X_real.fillna(0)
X_real_mean = X_real.fillna(data.mean(axis=0))
X_cat = X_cat.fillna('NA', axis=0).applymap(str)

Преобразовываем категориальные признаки с помощью One-hot encoding

In [7]:
encoder = DV(sparse = False)
X_cat_oh = encoder.fit_transform(X_cat.T.to_dict().values())

Строим модели.
Замечание: Далее будет использоваться количественные признаки с заменой пустых на среднее, так как модели дали эффективность выше на 5-7 пунктов. Этап сравнения был опущен, так как главная задача задания построение базовых моделей. 

1. Logistic Regression Baseline:

In [8]:
logit = LogisticRegression()

In [9]:
%%time
cv_scores_lr1 = cross_val_score(logit, 
                             np.concatenate((X_real_mean, X_cat_oh), axis = 1), 
                             labels, 
                             cv = 5, 
                            scoring='roc_auc')

CPU times: user 4min 34s, sys: 3min 29s, total: 8min 4s
Wall time: 6min 57s


In [10]:
%%time
cv_scores_lr2 = cross_val_score(logit, 
                             np.concatenate((X_real_mean, X_cat_oh), axis = 1), 
                             labels, 
                             cv = 5, 
                            scoring='f1')

CPU times: user 4min 24s, sys: 3min 25s, total: 7min 50s
Wall time: 6min 43s


In [12]:
print ('Результаты основной метрики ROC_AUC:',cv_scores_lr1, cv_scores_lr1.mean())

Результаты основной метрики ROC_AUC: [0.54305537 0.53693018 0.53084632 0.56968788 0.55469063] 0.5470420776901046


In [13]:
print ('Результаты вспомогательной метрики F1:', cv_scores_lr2, cv_scores_lr2.mean())

Результаты вспомогательной метрики F1: [0.02118644 0.04192872 0.04065041 0.03757829 0.02464066] 0.033196902708086604


2. Random Forest Baseline:

In [14]:
rf = RandomForestClassifier()

In [ ]:
%%time

cv_scores_rf1 = cross_val_score(rf, 
                             np.concatenate((X_real_mean, X_cat_oh), axis = 1), 
                             labels, 
                             cv = 5, 
                            scoring = 'roc_auc')

In [ ]:
%%time

cv_scores_rf2 = cross_val_score(rf, 
                             np.concatenate((X_real_mean, X_cat_oh), axis = 1), 
                             labels, 
                             cv = 5, 
                            scoring = 'f1')

In [ ]:
print ('Результаты основной метрики ROC_AUC:', cv_scores_rf1, cv_scores_rf1.mean())

In [ ]:
print ('Результаты вспомогательной метрики F1:', cv_scores_rf2, cv_scores_rf2.mean())

3. CatBoost Baseline:

In [22]:
cb = CatBoostClassifier()

In [23]:
%%time

cv_scores_cb1 = cross_val_score(cb, 
                             np.concatenate((X_real_mean, X_cat_oh), axis = 1), 
                             labels, 
                             cv = 5, 
                            scoring = 'roc_auc')

KeyboardInterrupt: 

In [ ]:
%%time

cv_scores_cb2 = cross_val_score(cb, 
                             np.concatenate((X_real_mean, X_cat_oh), axis = 1), 
                             labels, 
                             cv = 5, 
                            scoring = 'roc_auc')

In [ ]:
print ('Результаты основной метрики ROC_AUC:', cv_scores_cb1, cv_scores_cb1.mean())

In [ ]:
print ('Результаты вспомогательной метрики F1:', cv_scores_cb2, cv_scores_cb2.mean())